In [26]:
'''
Last Updated: 6/19/2019
Author: Joey Li
Overview: This is basically a fancy calculator that I use to figure out whether I'm on the right track with certain
computations. The scripts here are to facilitate easy generation of matrices for permutations on tensor products
and to be able to test constructions for things in the Schur basis.
'''

import numpy as np
from numpy import matrix
from numpy import kron
from math import sqrt
import scipy

# Bra and ket are just helper functions that build the appropriate vector out of a 
# bitstring specifying a tensor product
def bra(bitstring):
    return ket(bitstring).getT()
def ket(bitstring):
    # I don't think Python likes padded zeros so this is actually irrelevant
    if not isinstance(bitstring,str):
        bitstring = str(bitstring)
    vec = matrix('1')
    for i in range(len(bitstring)):
        if bitstring[i]=='0':
            vec = kron(vec,matrix('1;0'))
        elif bitstring[i]=='1':
            vec = kron(vec,matrix('0;1'))
    return vec
# A projector matrix from two bitstrings
def ketbra(a,b):
    return ket(a)*bra(b)


# Gives permutation matrix for perm string 'p' on n qubits in n-fold tensor space
def permMatrix(n, p):
    perm = makeDict(n,p)
    mat = np.zeros((2**n,2**n))
    for i in range(2**n):
        basisInt = bin(i)[2:].zfill(n)
        #print(basisInt)
        transformed = permuteString(basisInt, perm)
        #print(transformed)
        #print()
        mat = mat + ket(transformed)*bra(basisInt)
    return mat

# Given a permutation dictionary encoding the desired permutation, 
# it tells you how the permutation would transform
def permuteString(string, permDict):
    newString=''
    permDict = invertDict(permDict)
    for i in range(len(string)):
        newString += string[permDict[i+1]-1]
    return newString

# Not robust at all
# Swaps key and value, but assumes only one value per key
# Used because the action of a permutation on a string actually
# uses the inverse permutation not the original
def invertDict(dictionary):
    newDict = {}
    for i in dictionary.keys():
        newDict[dictionary[i]] = i
    return newDict

# Turns perm string 'p' into a dictionary for easier use
# Keys and values are all ints
def makeDict(n,p):
    key = {}
    for i in range(1,n+1):
        if str(i) in p:
            nextIndex = p.index(str(i))+1
            if nextIndex == len(p):
                nextIndex = 0
            key[i] = int(p[nextIndex])
        else:
            key[i] = i
    return key



schur = matrix([[1, 0, 0, 0, 0, 0, 0, 0],
               [0, 1/sqrt(3), 0, 0, 0, 0, 2/sqrt(6), 0],
                [0, 1/sqrt(3), 0, 0, 1/sqrt(2), 0, -1/sqrt(6), 0],
                [0,0,1/sqrt(3),0,0,-1/sqrt(2),0,-1/sqrt(6)],
                [0,1/sqrt(3),0,0,-1/sqrt(2),0,-1/sqrt(6),0],
                [0,0,1/sqrt(3),0,0,1/sqrt(2),0,-1/sqrt(6)],
                [0,0,1/sqrt(3),0,0,0,0,2/sqrt(6)],
                [0,0,0,1,0,0,0,0]
               ])

print(schur.round(decimals=2))



# This implements a (12) swap. It corresponds to some sort of CZ operation
'''
diag = matrix([
             [1,0,0,0,0,0,0,0],
             [0,1,0,0,0,0,0,0],
             [0,0,1,0,0,0,0,0],
             [0,0,0,1,0,0,0,0],
             [0,0,0,0,-1,0,0,0],
             [0,0,0,0,0,-1,0,0],
             [0,0,0,0,0,0,1,0],
             [0,0,0,0,0,0,0,1]
            ])
print(diag)
swapfirsttwo = schur*diag*schur**(-1)
print(swapfirsttwo.round(decimals=0))
'''


# This is what we would do in the schur basis to implement a (23) swap
# It corresponds to a single qubit rotation so that's pretty nice
schurswap = schur**(-1)*permMatrix(3,'23')*schur
print(schurswap.round(decimals=3))


# The code below came from trying to package the change of basis into schur basis, then
# the rotation into standard state, then change back into original basis all in one.
# However, it's not really necessary - just run the schur transform and do the simple 
# rotation like a normal person.
'''
# missing the ones with two disjoint transpositions because this was ad hoc coding
s4 = ['', '12', '13', '14', '23', '24', '34', '123', '124', '134', '132', '142', '143', '234', '243', '1234', '1243', '1324', '1342', '1423', '1432']

# Initialize first string of matrix
eqnMatrix = permMatrix(4,s4[i]).flatten()

# Add all the flattened matrices as columns in our equation matrix
for i in range(len(s4)):
    if i==0:
        continue
    eqnMatrix = np.vstack([eqnMatrix,permMatrix(4,s4[i]).flatten()])
    
#print(eqnMatrix)
    
    
# Add in the ones we missed above, ie, (12)(34), (13)(24), (14)(23)
tp1 = permMatrix(4,'12')*permMatrix(4,'34')
tp2 = permMatrix(4,'13')*permMatrix(4,'24')
tp3 = permMatrix(4,'14')*permMatrix(4,'23')

eqnMatrix = np.vstack([eqnMatrix, tp1.flatten()])
eqnMatrix = np.vstack([eqnMatrix, tp2.flatten()])
eqnMatrix = np.vstack([eqnMatrix, tp3.flatten()])

print(len(eqnMatrix))


# This is our target matrix
target = (sqrt(3/4)*ket('0001')+sqrt(1/12)*(ket('0010')+ket('0100')+ket('1000')))*1/sqrt(2)*(bra('0100')-bra('1000'))
target += sqrt(1/6)*(ket('0011')+ket('0101')+ket('1001')-ket('0110')-ket('1010')-ket('1100'))*1/2*(bra('0101')+bra('0110')-bra('1001')-bra('1010'))
target += (sqrt(1/12)*(ket('0111')+ket('1011')+ket('1101'))-sqrt(3/4)*ket('1110'))*1/sqrt(2)*(bra('0111')-bra('1011'))
target = target.flatten()
#print(target.flatten())

soln = np.linalg.lstsq(eqnMatrix.getT(), target.getT(), None)
print(soln)
'''

'''
projector = ketbra('000','000')+ketbra('001','001')+ketbra('010','010')+ ketbra('011','011')+ketbra('110','100')+ketbra('111','101')+ketbra('100','110')+ketbra('101','111')
realprojector = schur*projector*np.linalg.inv(schur)
print(realprojector)
print()
print(permMatrix(3,'13')+permMatrix(3,'123')+permMatrix(3,'1')+permMatrix(3,'12')+permMatrix(3,'23')+permMatrix(3,'132'))

print(np.vectorize(realprojector))
'''


[[ 1.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.58  0.    0.    0.    0.    0.82  0.  ]
 [ 0.    0.58  0.    0.    0.71  0.   -0.41  0.  ]
 [ 0.    0.    0.58  0.    0.   -0.71  0.   -0.41]
 [ 0.    0.58  0.    0.   -0.71  0.   -0.41  0.  ]
 [ 0.    0.    0.58  0.    0.    0.71  0.   -0.41]
 [ 0.    0.    0.58  0.    0.    0.    0.    0.82]
 [ 0.    0.    0.    1.    0.    0.    0.    0.  ]]
[[ 1.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     1.     0.     0.     0.     0.    -0.     0.   ]
 [ 0.     0.     1.     0.     0.    -0.     0.    -0.   ]
 [ 0.     0.     0.     1.     0.     0.     0.     0.   ]
 [ 0.    -0.     0.     0.     0.5    0.     0.866  0.   ]
 [ 0.     0.     0.     0.     0.     0.5    0.     0.866]
 [ 0.    -0.     0.     0.     0.866  0.    -0.5    0.   ]
 [ 0.     0.    -0.     0.     0.     0.866  0.    -0.5  ]]


"\nprojector = ketbra('000','000')+ketbra('001','001')+ketbra('010','010')+ ketbra('011','011')+ketbra('110','100')+ketbra('111','101')+ketbra('100','110')+ketbra('101','111')\nrealprojector = schur*projector*np.linalg.inv(schur)\nprint(realprojector)\nprint()\nprint(permMatrix(3,'13')+permMatrix(3,'123')+permMatrix(3,'1')+permMatrix(3,'12')+permMatrix(3,'23')+permMatrix(3,'132'))\n\nprint(np.vectorize(realprojector))\n"